# GWAS Tutorial — Part 3: Association Analysis & Results

With a clean, QC-filtered dataset in hand, we can now run the genome-wide association study and interpret the results. This notebook covers:

1. **Population stratification** — PCA to identify ancestry and use as covariates
2. **Linear regression** — test each variant for association with caffeine consumption
3. **Results visualization** — Manhattan plot, QQ plot, genomic inflation
4. **Top hits** — examine the most significant variants

> **Prerequisites:** Run [Part 2](https://colab.research.google.com/github/gosborcz/winterschool-gwas-tutorial/blob/main/02_quality_control.ipynb) first so that `data/1kg_qc.mt` exists, or use the Setup cell below to run the full pipeline from scratch.

## Setup

This cell installs Hail, downloads data, and re-runs the QC pipeline if `data/1kg_qc.mt` is not found.

In [ ]:
!apt-get install -y openjdk-11-jdk-headless -q
!pip install hail -q

import hail as hl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
from pprint import pprint

hl.init()
from hail.plot import show
hl.plot.output_notebook()

import os
os.makedirs('data', exist_ok=True)

if not os.path.exists('data/1kg_qc.mt'):
    print('QC MatrixTable not found — running full pipeline...')
    if not os.path.exists('data/1kg.mt'):
        hl.utils.get_1kg('data/')
        hl.import_vcf('data/1kg.vcf.bgz').write('data/1kg.mt', overwrite=True)
    table = hl.import_table('data/1kg_annotations.txt', impute=True).key_by('Sample')
    mt = hl.read_matrix_table('data/1kg.mt')
    mt = mt.annotate_cols(pheno=table[mt.s])
    mt = hl.sample_qc(mt)
    mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
    ab = mt.AD[1] / hl.sum(mt.AD)
    filter_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                 (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                 (mt.GT.is_hom_var() & (ab >= 0.9)))
    mt = mt.filter_entries(filter_ab)
    mt = hl.variant_qc(mt)
    mt = mt.filter_rows((mt.variant_qc.AF[1] > 0.01) & (mt.variant_qc.p_value_hwe > 1e-6))
    mt.write('data/1kg_qc.mt', overwrite=True)
else:
    # Load QC-filtered data and re-attach phenotypes
    table = hl.import_table('data/1kg_annotations.txt', impute=True).key_by('Sample')
    mt = hl.read_matrix_table('data/1kg_qc.mt')
    if 'pheno' not in mt.col:
        mt = mt.annotate_cols(pheno=table[mt.s])

print('Ready: %d variants x %d samples' % mt.count())

## 1. Population Stratification via PCA

The 1000 Genomes dataset includes individuals from multiple ancestral populations (AFR, AMR, EAS, EUR, SAS). If we don't account for this **population structure**, variants that differ in frequency across populations will appear associated with our phenotype — not because they affect it, but because population labels correlate with both genotype and phenotype.

We use **Principal Component Analysis (PCA)** on the genotype matrix to capture axes of genetic variation that reflect ancestry. These PCs are then included as covariates in the regression model.

`hl.hwe_normalized_pca()` normalizes each variant by its Hardy-Weinberg expected variance before computing PCs — this prevents high-frequency variants from dominating.

In [ ]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)

pprint(eigenvalues[:5])  # top 5 eigenvalues — how much variance each PC explains

In [ ]:
# Attach PC scores to sample columns
mt = mt.annotate_cols(scores=pcs[mt.s].scores)
pcs.show(3, width=100)

In [ ]:
# PCA scatter plot coloured by super-population — clear ancestry clusters expected
p = hl.plot.scatter(
    mt.scores[0], mt.scores[1],
    label=mt.pheno.SuperPopulation,
    title='PCA — PC1 vs PC2 coloured by Super-Population',
    xlabel='PC1', ylabel='PC2'
)
show(p)

In [ ]:
# PC2 vs PC3 — additional structure
p = hl.plot.scatter(
    mt.scores[1], mt.scores[2],
    label=mt.pheno.SuperPopulation,
    title='PCA — PC2 vs PC3 coloured by Super-Population',
    xlabel='PC2', ylabel='PC3'
)
show(p)

## 2. GWAS Without Population Covariates (Naive)

As a baseline, we first run linear regression without any covariates. This will show inflated results because population structure is not accounted for.

In [ ]:
gwas_naive = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0]  # intercept only
)
gwas_naive.row.describe()

In [ ]:
p = hl.plot.qq(gwas_naive.p_value, title='QQ Plot (naive, no covariates)')
show(p)

In [ ]:
p = hl.plot.manhattan(gwas_naive.p_value, title='Manhattan Plot (naive, no covariates)')
show(p)

### Genomic Inflation Factor (λ)

The **genomic inflation factor (λ)** measures how much the observed test statistics are inflated relative to the null expectation. It is computed as the ratio of the median observed chi-squared statistic to the expected median under the null (0.4549).

- λ ≈ 1.0 → well-calibrated test (no inflation)
- λ > 1.05 → inflation, likely due to population stratification or cryptic relatedness
- λ < 1.0 → deflation (possible over-correction)

In [ ]:
from scipy import stats

# Collect p-values and compute lambda
pvals_naive = gwas_naive.p_value.collect()
chi2_naive = [stats.chi2.ppf(1 - p, df=1) for p in pvals_naive if p is not None and 0 < p < 1]
lambda_naive = np.median(chi2_naive) / stats.chi2.ppf(0.5, df=1)
print('Genomic inflation factor (naive):  lambda = %.3f' % lambda_naive)

## 3. GWAS With Population Covariates (Corrected)

Now we include sex and the top 3 PCs as covariates. This controls for population stratification and should bring λ close to 1.0.

In [ ]:
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]]
)
print('GWAS complete: %d variants tested' % gwas.count())

In [ ]:
pvals = gwas.p_value.collect()
chi2 = [stats.chi2.ppf(1 - p, df=1) for p in pvals if p is not None and 0 < p < 1]
lambda_gc = np.median(chi2) / stats.chi2.ppf(0.5, df=1)
print('Genomic inflation factor (corrected): lambda = %.3f' % lambda_gc)

In [ ]:
p = hl.plot.qq(gwas.p_value, title='QQ Plot (corrected: sex + PC1-3)')
show(p)

In [ ]:
p = hl.plot.manhattan(gwas.p_value, title='Manhattan Plot (corrected: sex + PC1-3)')
show(p)

### QQ Plot with λ Annotation (matplotlib)

We redraw the QQ plot using matplotlib so we can annotate it with the genomic inflation factor.

In [ ]:
valid_pvals = np.array([p for p in pvals if p is not None and 0 < p < 1])
n = len(valid_pvals)

obs = -np.log10(np.sort(valid_pvals)[::-1])  # observed -log10(p), descending
exp = -np.log10(np.linspace(1/n, 1, n)[::-1])  # expected under null

fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(exp, obs, s=6, alpha=0.6, color='steelblue')
max_val = max(obs.max(), exp.max()) * 1.05
ax.plot([0, max_val], [0, max_val], color='red', linewidth=1, linestyle='--', label='y = x (null)')
ax.set_xlabel('Expected -log10(p)', fontsize=12)
ax.set_ylabel('Observed -log10(p)', fontsize=12)
ax.set_title('QQ Plot — GWAS Results', fontsize=14)
ax.text(0.05, 0.95, 'lambda = %.3f' % lambda_gc, transform=ax.transAxes,
        fontsize=12, va='top', color='darkred')
ax.legend()
plt.tight_layout()
plt.show()

## 4. Top Hits

Let's examine the most significant variants. Since this phenotype is simulated, we don't expect any true positives — but this workflow would identify real hits with real phenotype data.

In [ ]:
# Sort by p-value and show top 20
gwas_sorted = gwas.order_by(gwas.p_value)
gwas_sorted.select(
    'p_value', 'beta', 'standard_error', 't_stat'
).show(20, width=120)

In [ ]:
# Collect top hits for a forest-plot style view of effect sizes
top = gwas.order_by(gwas.p_value).take(20)

betas = np.array([r.beta for r in top])
ses = np.array([r.standard_error for r in top])
labels = ['%s:%s/%s' % (r.locus.contig, r.locus.position, r.alleles[1]) for r in top]

fig, ax = plt.subplots(figsize=(8, 7))
y = np.arange(len(top))
ax.errorbar(betas, y, xerr=1.96 * ses, fmt='o', color='steelblue',
            capsize=3, linewidth=1, markersize=5, label='Beta +/- 95% CI')
ax.axvline(x=0, color='grey', linestyle='--', linewidth=1)
ax.set_yticks(y)
ax.set_yticklabels(labels, fontsize=8)
ax.set_xlabel('Effect Size (beta)', fontsize=12)
ax.set_title('Top 20 Variants — Effect Size (beta) with 95% CI', fontsize=13)
ax.invert_yaxis()
plt.tight_layout()
plt.show()

## Summary

In this notebook you:

- ✅ Ran PCA to visualize population structure and generate ancestry covariates
- ✅ Ran a naive GWAS (no covariates) and observed inflation (λ > 1)
- ✅ Ran a corrected GWAS with sex + top 3 PCs as covariates
- ✅ Computed the genomic inflation factor (λ) before and after correction
- ✅ Visualized results with Manhattan and QQ plots
- ✅ Examined the top associated variants and their effect sizes

---

**What next?**
- Apply to a real phenotype with genuine genetic architecture
- Add more covariates (age, batch, site)
- Fine-map significant loci using conditional analysis
- Annotate top hits with gene and regulatory information (e.g., gnomAD, GTEx)